In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import BorderlineSMOTE

In [15]:
#读取数据
X_train = pd.read_csv('../preprocessed_data/X_train.csv',header=None).values
X_test = pd.read_csv('../preprocessed_data/X_test.csv',header=None).values

In [16]:
y_train = pd.read_csv('../preprocessed_data/y_train.csv',header=None)
y_test = pd.read_csv('../preprocessed_data/y_test.csv',header=None)

In [17]:
DoS = ['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm']
Probe = ['ipsweep','mscan','nmap','portsweep','saint','satan']
R2L = ['ftp_write','guess_passwd','httptunnel','imap','multihop','phf','sendmail','snmpgetattack','spy','snmpguess','warezclient','warezmaster','xlock','xsnoop','named']
U2R = ['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm']

In [18]:
#将y_train和y_test中的异常子类映射为四大类
y_train = y_train.replace(DoS,'DoS')
y_train = y_train.replace(Probe,'Probe')
y_train = y_train.replace(R2L,'R2L')
y_train = y_train.replace(U2R,'U2R')

y_test = y_test.replace(DoS,'DoS')
y_test = y_test.replace(Probe,'Probe')
y_test = y_test.replace(R2L,'R2L')
y_test = y_test.replace(U2R,'U2R')

In [19]:
#查看各类别样本数量
print(y_train[0].value_counts())
print(y_test[0].value_counts())
#确保每个异常子类都被映射为四大类

normal    67343
DoS       45927
Probe     11656
R2L         995
U2R          52
Name: 0, dtype: int64
normal    9711
DoS       7460
R2L       2885
Probe     2421
U2R         67
Name: 0, dtype: int64


In [20]:
#对训练集进行随机复制过采样
#from imblearn.over_sampling import RandomOverSampler
#ros = RandomOverSampler(random_state=42)
#X_train, y_train = ros.fit_resample(X_train, y_train)

In [21]:
#对训练集进行BorderlineSMOTE随机过采样(2'40")
#borderline_smote = BorderlineSMOTE(random_state=42)
#X_train, y_train = borderline_smote.fit_resample(X_train, y_train)
#试过了,效果不好

In [22]:
#查看各类别样本数量
print(y_train[0].value_counts())

normal    67343
DoS       45927
Probe     11656
R2L         995
U2R          52
Name: 0, dtype: int64


In [23]:
#对y_train和y_test使用sklearn的to_categorical函数进行one-hot编码
from sklearn.preprocessing import OneHotEncoder
#合并y_train和y_test
y = np.concatenate((y_train,y_test))
enc = OneHotEncoder()
y = enc.fit_transform(y.reshape(-1,1)).toarray()
y_train = y[:len(y_train)]
y_test = y[len(y_train):]

In [24]:
#划分验证集
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [25]:
#使用随机森林进行多分类
rf = RandomForestClassifier(n_estimators=100,random_state=42)
rf.fit(X_train,y_train)
#在验证集上进行预测
y_pred_val = rf.predict(X_val)
#显示准确率
print('accuracy_score:',accuracy_score(y_val,y_pred_val))

accuracy_score: 0.9988489779718198


In [26]:
#在验证集上进行预测
y_pred = rf.predict(X_test)
#显示准确率
print('accuracy_score:',accuracy_score(y_test,y_pred))

accuracy_score: 0.7365596167494677
